In [1]:
import os
import pandas as pd
import numpy as np


path_cwd = os.getcwd()
path_X = path_cwd+"/ML4G_Project_1_Data/X_data"
path_y=path_cwd+"/ML4G_Project_1_Data/CAGE-train"
X_folders=os.listdir(path_X)
y_folders=os.listdir(path_y)
#pd.read_csv(path_test, sep='\t')
X_folders,y_folders

(['DNase-bed',
  'H3K27ac-bed',
  'H3K27me3-bed',
  'H3K36me3-bed',
  'H3K4me1-bed',
  'H3K4me3-bed',
  'H3K9me3-bed'],
 ['X1_train_info.tsv',
  'X1_train_y.tsv',
  'X1_val_info.tsv',
  'X1_val_y.tsv',
  'X2_train_info.tsv',
  'X2_train_y.tsv',
  'X2_val_info.tsv',
  'X2_val_y.tsv',
  'X3_test_info.tsv'])

For every gene collector:
We will create a numpy 2d array for every feature of size gene per window
and then we will stack them.

In [2]:
window_size=1000
binary=True

def window(start,end,window_size):
    r=int(window_size/2)
    m=(start+end)/2
    return (m-r,m+r)


for y_file_name in y_folders:
    if not 'info' in y_file_name: continue
    collector_name=y_file_name[:-9]
    cell_line=y_file_name[:2]
    y_file_path= path_y+ '/' + y_file_name
    y_pd=pd.read_csv(y_file_path, sep='\t')
    n_genes=y_pd.shape[0]
    for X_file_name in X_folders:
        X_file_path=path_X+ '/' + X_file_name
        variable_name=X_file_name[:-4]
        X_pd=pd.read_csv(X_file_path +'/'+cell_line + '.bed', sep='\t',header=None)
        X= collector_name + '_' +variable_name
        vars().__setitem__(X, np.zeros((n_genes,window_size),dtype=int))
        for idx, yrow in y_pd.iterrows():
            chr=yrow.chr
            start_pos, end_pos= window(yrow.TSS_start,yrow.TSS_end,window_size)
            exit





In [3]:
collector_name

'X3_test'

# Draft

In [3]:
path_test = path_data+"/CAGE-train/X1_train_info.tsv"   # X3_test_info.tsv ; TODO
test_genes =

In [63]:

myStr='ciao'
vars().__setitem__(myStr, 97)
ciao

97

In [85]:
path_cwd = os.getcwd()
path_data = path_cwd+"/ML4G_Project_1_Data/X_Data/DNase-bed/X1.bed"
test_bed=pd.read_csv(path_data, sep='\t',header=None)
for i, row in test_bed.iterrows():
    print(row)
    break

0         chr1
1       181405
2       181555
3            .
4            0
5            .
6    12.379543
7           -1
8           -1
9           75
Name: 0, dtype: object


In [80]:
path_data,X_file_path

('C:\\Users\\gugli\\Documents\\GitHub\\ml4g_2023/ML4G_Project_1_Data/X_Data/DNase-bed/X1.bed',
 'C:\\Users\\gugli\\Documents\\GitHub\\ml4g_2023/ML4G_Project_1_Data/X_data/DNase-bed')

§B